## Calibrating the Ornstein-Uhlenbeck (Vasicek) model


### The OU Equation

The stochastic differential equation (SDE) for the Ornstein-Uhlenbeck process is given by:

$$ {dS}_{t}= \alpha(\mu - {S}_{t}){dt} + \sigma{dW}_{t}$$

with $\alpha$ the mean reversion rate, $\mu$ the long-term mean, $\sigma$ the volatility

### Calibration using least squres regression 

The relationship between consecutive observation ${S}_{i}, {S}_{i+1}$ in linear with a iid normal random term $\epsilon$:

$$ {S}_{i+1} = {aS}_{i} + {b} + {\epsilon}$$

The relationship between the linear fit and model parameters is given by:

$$ {a} = {e}^{{-\alpha} {\delta}} $$

$$ {b} = {\mu}(1- {e}^{{-\alpha} {\delta}}) $$

$$ {sd}({\epsilon}) = {\sigma} \sqrt{\frac{1- {e}^{{-2\alpha} {\delta}}}{2\alpha}} $$

rewriting these equations gives,

$$ \alpha = - \frac{\ln{a}}{\delta} $$

$$ \mu = \frac{b}{1-a}$$

$$ \sigma = {sd}(\epsilon) \sqrt{\frac{-2\ln{a}}{\delta(1-a^2)}} $$


In [8]:
#import sample data 
import pandas as pd 
import numpy as np
df = pd.read_csv('sample_set.csv')

In [9]:
# import statsmodels.api to utilise their OLSregression function
import statsmodels.api as sm
# using sample data, create X and y values 
X = df['t1']
y = df['orig']

#add a constant to the model show as 'const' in results
X = sm.add_constant(X)

#fit the model
results = sm.OLS(y,X).fit()

#display regression results
print (results.summary())

                            OLS Regression Results                            
Dep. Variable:                   orig   R-squared:                       0.905
Model:                            OLS   Adj. R-squared:                  0.905
Method:                 Least Squares   F-statistic:                 1.042e+05
Date:                Thu, 11 Feb 2021   Prob (F-statistic):               0.00
Time:                        12:43:43   Log-Likelihood:                 48802.
No. Observations:               10934   AIC:                        -9.760e+04
Df Residuals:                   10932   BIC:                        -9.759e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0003   3.41e-05    -10.231      0.0

In [10]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~ FUNCTION: OU_calibration ~~~~~~~
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# Inputs:
#     - df - dataframe containing the prices for each asset 
#     - spread - list containing thename of each asset in the spread 
# The function:
# Using user inputs the fnction will determine regression results, 
# to then be used to calibrate the OU process shown in equations above
    
    
def OU_calibration(df, spread):
    # get namefor asset A & B
    A = spread[0]
    B = spread[1]
    
    # Get log spread
    df['spread'] = np.log((df[A] / df[B]))
    # shift the spread
    df['shift'] = df.spread.shift()
    df.loc[0, 'shift'] = 0
    
    # using sample data, create X and y values 
    X = df['shift']
    y = df['spread']
    # add constant 
    X = sm.add_constant(X)
    
    # fit using OLS regression
    results = sm.OLS(y,X).fit()
    
    #store order
    order = A, B
    
    # check results if results.params[0] < 0, flip the order of the spread
    
    if results.params[0] < 0:
        df['spread'] = np.log((df[B] / df[A]))
        df['shift'] = df.spread.shift()
        df.loc[0, 'shift'] = 0
        X = df['shift']
        y = df['spread']
        X = sm.add_constant(X)
        results = sm.OLS(y,X).fit()
        order = B, A
        
    # return results 
    return results.params[0], results.params[1], results.bse[1], order

In [11]:
OU_calibration(df, ('CO1', 'CO2'))

(0.00035012565107297875,
 0.9513944476006523,
 0.0029479606003722,
 ('CO2', 'CO1'))

In [12]:
#Validation of daily results
# as seen in the validation section of the report 
import numpy as np
a = 0.9998337
b=0.0012
sde=0.0123

alpha = -np.log(a)
mu = b/(1-a)
sigma =sde*np.sqrt((-2*np.log(a))/(1-a*a))

print ('alpha: ', alpha)
print ('mu: ',mu)
print('sigma: ', sigma)

alpha:  0.0001663138293781909
mu:  7.215874924836692
sigma:  0.012301022844224662


In [13]:
#Validation of annual results
# as seen in the validation section of the report 
import numpy as np
a = 0.9515
b=0.3451
sde=0.1906

alpha = -np.log(a)
mu = b/(1-a)
sigma =sde*np.sqrt((-2*np.log(a))/(1-a*a))

print ('alpha: ', alpha)
print ('mu: ',mu)
print('sigma: ', sigma)

alpha:  0.04971559224593286
mu:  7.1154639175257755
sigma:  0.19535703522704506
